In [1]:
import deepcut

In [2]:
import re

In [3]:
import numpy as np

In [4]:
from nltk import FreqDist, precision, recall, f_measure, NaiveBayesClassifier
from nltk.classify import apply_features
from nltk.classify import util

In [5]:
from sklearn.model_selection import KFold
import collections, itertools

In [6]:
data_pos = [(line.strip(), 'pos') for line in open("./DataSet/pos.txt", 'r')]
data_neg = [(line.strip(), 'neg') for line in open("./DataSet/neg.txt", 'r')]

In [7]:
def split_words (sentence):
    return deepcut.tokenize(''.join(sentence.lower().split()))

In [8]:
sentences = [(split_words(sentence), sentiment) for (sentence, sentiment) in data_pos + data_neg]

In [9]:
sentences

[(['ดี'], 'pos'),
 (['ขอบคุณ'], 'pos'),
 (['สู้', 'เสมอ'], 'pos'),
 (['ผ่าน'], 'pos'),
 (['โคตรสนุก'], 'pos'),
 (['สุดยอด'], 'pos'),
 (['น่ารัก'], 'pos'),
 (['ขอบคุณ', 'ทุก'], 'pos'),
 (['คง', 'คิด', 'ถึง'], 'pos'),
 (['เก็บ', 'เวลา', 'ที่', 'ดี', 'ให้', 'เรา', 'ได้', 'จดจำ'], 'pos'),
 (['ทำ', 'ข้อ', 'สอบ', 'ให้', 'ได้'], 'pos'),
 (['สู้'], 'pos'),
 (['รอย', 'ยิ้ม', 'ใน', 'ความทรงจำ'], 'pos'),
 (['ทุก', 'นี้', 'ฉัน', 'คิด', 'ถึง'], 'pos'),
 (['ความทรงจำ', 'ที่', 'ดี', 'เหลือเกิน'], 'pos'),
 (['รัก', 'มาก', 'คิด', 'ถึง', 'มาก'], 'pos'),
 (['ดีใจ'], 'pos'),
 (['รัก'], 'pos'),
 (['ห่วง'], 'pos'),
 (['รัก', 'แฟน', 'ขี้', 'บ่น', 'จัง'], 'pos'),
 (['ขอบคุณ', 'ผู้ใจ', 'ดี'], 'pos'),
 (['มิตรภาพ', 'ดี'], 'pos'),
 (['ที่รัก'], 'pos'),
 (['ชอบ', 'จัง'], 'pos'),
 (['ความทรงจำ', 'ที่', 'ยัง', 'สวยงาม'], 'pos'),
 (['ช่วง',
   'เวลา',
   'ที่',
   'รู้จัก',
   'ถึง',
   'จะ',
   'ไม่',
   'นาน',
   'แต่',
   'คือ',
   'ช่วง',
   'ที่',
   'ดี',
   'ที่สุด'],
  'pos'),
 (['เปลี่ยน', 'ไป', 'รัก', 'รัก

In [10]:
def get_words_in_sentences(sentences):
    all_words = []
    for (words, sentiment) in sentences:
        all_words.extend(words)
    return all_words

In [11]:
wordlist = get_words_in_sentences(sentences)

In [17]:
wordlist

['ดี',
 'ขอบคุณ',
 'สู้',
 'เสมอ',
 'ผ่าน',
 'โคตรสนุก',
 'สุดยอด',
 'น่ารัก',
 'ขอบคุณ',
 'ทุก',
 'คง',
 'คิด',
 'ถึง',
 'เก็บ',
 'เวลา',
 'ที่',
 'ดี',
 'ให้',
 'เรา',
 'ได้',
 'จดจำ',
 'ทำ',
 'ข้อ',
 'สอบ',
 'ให้',
 'ได้',
 'สู้',
 'รอย',
 'ยิ้ม',
 'ใน',
 'ความทรงจำ',
 'ทุก',
 'นี้',
 'ฉัน',
 'คิด',
 'ถึง',
 'ความทรงจำ',
 'ที่',
 'ดี',
 'เหลือเกิน',
 'รัก',
 'มาก',
 'คิด',
 'ถึง',
 'มาก',
 'ดีใจ',
 'รัก',
 'ห่วง',
 'รัก',
 'แฟน',
 'ขี้',
 'บ่น',
 'จัง',
 'ขอบคุณ',
 'ผู้ใจ',
 'ดี',
 'มิตรภาพ',
 'ดี',
 'ที่รัก',
 'ชอบ',
 'จัง',
 'ความทรงจำ',
 'ที่',
 'ยัง',
 'สวยงาม',
 'ช่วง',
 'เวลา',
 'ที่',
 'รู้จัก',
 'ถึง',
 'จะ',
 'ไม่',
 'นาน',
 'แต่',
 'คือ',
 'ช่วง',
 'ที่',
 'ดี',
 'ที่สุด',
 'เปลี่ยน',
 'ไป',
 'รัก',
 'รัก',
 'จน',
 'หมด',
 'หัวใจ',
 'ดีใจ',
 'ด้วย',
 'ปลอด',
 'ภัย',
 'ทรง',
 'พระเจริญ',
 'ใจเย็น',
 'ดูแล',
 'ตัว',
 'เอง',
 'ด้วย',
 'ขอบคุณมิตรภาพ',
 'ดี',
 'เพื่อน',
 'ที่',
 'ดี',
 'โอกาศ',
 'ดี',
 'แบบ',
 'นี้',
 'ห้าม',
 'พลาด',
 'ขอบคุณ',
 'ง่าย',
 'มอง',
 'โลก',
 'ใน',

In [12]:
def get_word_features(wordlist):
    wordlist = FreqDist(wordlist)
    word_features = [word[0] for word in wordlist.most_common()]
    return word_features

In [18]:
word_features = get_word_features(wordlist)
word_features

['ไม่',
 'ความ',
 'ดี',
 'ที่',
 'ให้',
 'ทำ',
 'ไป',
 'รัก',
 'น่า',
 'นี้',
 'มาก',
 'ได้',
 'ใน',
 'ตัว',
 'การ',
 'ที่สุด',
 'เอง',
 'เสีย',
 'อยู่',
 'อย่าง',
 'สิ่ง',
 'จะ',
 'ด้วย',
 'จาก',
 'คิด',
 'ถึง',
 'โลก',
 'รู้สึก',
 'อยาก',
 'ของ',
 'ตาย',
 'มัน',
 'ทุก',
 'ยัง',
 'หมด',
 'เห็น',
 'ว่า',
 'ผิด',
 'ชอบ',
 'เกิน',
 'ๆ',
 'เรา',
 'ใคร',
 'มา',
 'ขึ้น',
 'สุข',
 'จริง',
 'อย่า',
 'อะไร',
 'เธอ',
 'เหมือน',
 'ค่า',
 'ชีวิต',
 'รู้',
 'เจ็บ',
 'ฉัน',
 'มอง',
 'กลัว',
 'กว่า',
 'หวัง',
 'ออก',
 'และ',
 'โดน',
 'ขอบคุณ',
 'สู้',
 'สอบ',
 'ยิ้ม',
 'คือ',
 'จน',
 'ง่าย',
 'ต่อ',
 'ก็',
 'เดิม',
 'งาน',
 'ร้องไห้',
 'อีก',
 'เรื่อง',
 'เวลา',
 'ความทรงจำ',
 'แบบ',
 'กำลังใจ',
 'เพราะ',
 'ขอ',
 'กลับ',
 'ไว้',
 'แพ้',
 'ตาม',
 'ร้อน',
 'หลอก',
 'เสมอ',
 'ต้อง',
 'หาย',
 'กิน',
 'เข้า',
 'พูด',
 'เจอ',
 'พอ',
 'น้อย',
 'ไง',
 'ไร้',
 'ข้อ',
 'ขี้',
 'จัง',
 'รู้จัก',
 'เพื่อน',
 'พลาด',
 'ขนาด',
 'อร่อย',
 'นั้น',
 'พบ',
 'กำลัง',
 'ฝัน',
 'ตก',
 'ทิ้ง',
 'ก่อน',
 'ลา',
 'ทัน',
 'ย

In [14]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [22]:
features_data = np.array(sentences)
features_data

array([[list(['ดี']), 'pos'],
       [list(['ขอบคุณ']), 'pos'],
       [list(['สู้', 'เสมอ']), 'pos'],
       ...,
       [list(['อีเหี้ย']), 'neg'],
       [list(['เจ็บ', 'แสบ']), 'neg'],
       [list(['ลา', 'ก่อน']), 'neg']], dtype=object)

In [24]:
k_fold = KFold(n_splits=10, random_state=1992, shuffle=True)
word_features = None
accuracy_scores = []
for train_set, test_set in k_fold.split(features_data):
    word_features = get_word_features(get_words_in_sentences(features_data[train_set].tolist()))
    train_features = apply_features(extract_features, features_data[train_set].tolist())
    test_features = apply_features(extract_features, features_data[test_set].tolist())
    classifier = NaiveBayesClassifier.train(train_features)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
    for i, (feats, label) in enumerate(test_features):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
    accuracy_score = util.accuracy(classifier, test_features)
    print('train: {} test: {}'.format(len(train_set), len(test_set)))
    print('=================== Results ===================')
    print('Accuracy {:f}'.format(accuracy_score))
    print('            Positive     Negative')
    print('F1         [{:f}     {:f}]'.format(
        f_measure(refsets['pos'], testsets['pos']),
        f_measure(refsets['neg'], testsets['neg'])
    ))
    print('Precision  [{:f}     {:f}]'.format(
        precision(refsets['pos'], testsets['pos']),
        precision(refsets['neg'], testsets['neg'])
    ))
    print('Recall     [{:f}     {:f}]'.format(
        recall(refsets['pos'], testsets['pos']),
        recall(refsets['neg'], testsets['neg'])
    ))
    print('===============================================\n')

train: 956 test: 107
=================== Results ===================
Accuracy 0.728972
            Positive     Negative
F1         [0.602740     0.794326]
Precision  [0.916667     0.674699]
Recall     [0.448980     0.965517]

train: 956 test: 107
=================== Results ===================
Accuracy 0.785047
            Positive     Negative
F1         [0.741573     0.816000]
Precision  [0.804878     0.772727]
Recall     [0.687500     0.864407]

train: 956 test: 107
=================== Results ===================
Accuracy 0.747664
            Positive     Negative
F1         [0.666667     0.796992]
Precision  [0.964286     0.670886]
Recall     [0.509434     0.981481]

train: 957 test: 106
=================== Results ===================
Accuracy 0.735849
            Positive     Negative
F1         [0.611111     0.800000]
Precision  [0.846154     0.700000]
Recall     [0.478261     0.933333]

train: 957 test: 106
=================== Results ===================
Accuracy 0.801887
     